In [1]:
import pandas as pd
import numpy as np


df = pd.read_csv(
    "/home/jupyter-xueyin/GogoLeoTaki/data/processed/processed_price_data_2023Q1.csv"
)


df['Price2'] = df['Price'].str.replace('Rs. ', '', regex=False).str.replace(',', '').astype(float)


def extract_days(val):
    if pd.isna(val) or val == 'N.A.':
        return np.nan
    val = str(val).lower()
    if 'day' in val:
        try:
            return int(val.split()[0])
        except:
            return np.nan
    return np.nan

df['Days2'] = df['Validity'].apply(extract_days)


df[['Tag1', 'Tag2']] = df['Tags'].str.extract(r'([^ ]+)\s+(.*)')


df['clean_price_per_day'] = df['Price2'] / df['Days2']

# 计算平均价格
df_price_long = df[['Circle', 'Tag1', 'clean_price_per_day']].copy()
df_price_long.columns = ['circle', 'company', 'avg_price']
df_price_long = df_price_long.groupby(['circle', 'company'], as_index=False)['avg_price'].mean()

# 保存为 Excel
df_price_long.to_excel(
    "/home/jupyter-xueyin/GogoLeoTaki/data/processed/price_final_2023Q1.xlsx",
    index=False
)